In [105]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [106]:
base_dir = os.path.abspath('datafiles')
categories = ['Cheetah', 'Crocodile', 'Elephants', 'Tiger']
data = []
for category in categories:
    category_path = os.path.join(base_dir, category)
    if os.path.isdir(category_path):
        for file_name in os.listdir(category_path):
            # print(file_name)
            data.append([os.path.join(category_path, file_name), category])

df = pd.DataFrame(data, columns=['file_path', 'label'])
# print(df)

In [107]:
df['label'] = df['label'].astype('category').cat.codes
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
test_df

,file_path,label
1448,d:\projects-activities\Machine-Learning-LS-24\...,3
168,d:\projects-activities\Machine-Learning-LS-24\...,0
220,d:\projects-activities\Machine-Learning-LS-24\...,0
1643,d:\projects-activities\Machine-Learning-LS-24\...,3
344,d:\projects-activities\Machine-Learning-LS-24\...,0
...,...,...
188,d:\projects-activities\Machine-Learning-LS-24\...,0
350,d:\projects-activities\Machine-Learning-LS-24\...,0
1460,d:\projects-activities\Machine-Learning-LS-24\...,3
944,d:\projects-activities\Machine-Learning-LS-24\...,1


In [108]:
def load_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [64, 64])
    img = img / 255.0
    return img, label

def augment_image(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, max_delta=0.1)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices((train_df['file_path'].values, train_df['label'].values))
train_dataset = train_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.map(augment_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_df['file_path'].values, test_df['label'].values))
test_dataset = test_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [109]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(categories), activation='softmax')  # Number of categories
])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Users\aditi\miniconda3\envs\project\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_56 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_56 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_57 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_58 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,455,108 (9.37 MB)

 Trainable params: 2,455,108 (9.37 MB)

 Non-trainable params: 0 (0.00 B)

In [110]:
history = model.fit(
    train_dataset,
    epochs=200,
    validation_data=test_dataset
)

Epoch 1/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - accuracy: 0.2966 - loss: 1.3664 - val_accuracy: 0.3413 - val_loss: 1.2874
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.4336 - loss: 1.2552 - val_accuracy: 0.5210 - val_loss: 1.1969
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.4725 - loss: 1.1921 - val_accuracy: 0.5030 - val_loss: 1.0998
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - accuracy: 0.5275 - loss: 1.0806 - val_accuracy: 0.5449 - val_loss: 1.0518
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.5619 - loss: 1.0341 - val_accuracy: 0.5269 - val_loss: 1.0449
Epoch 6/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.5791 - loss: 0.9970 - val_accuracy: 0.5629 - val_loss: 0.9741
Epoch 7/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.6146 - loss: 0.9457 - val_accuracy: 0.6108 - val_loss: 0.9452
Epoch 8/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.6261 - loss: 0.8786 - val_accuracy: 

In [111]:
def test_model(model, test_dataset):
    test_loss, test_accuracy = model.evaluate(test_dataset)
    test_accuracy_percentage = test_accuracy * 100
    
    print(f'Test Accuracy: {test_accuracy_percentage:.2f}%')
    print(f'Test Loss: {test_loss}')


test_model(model, test_dataset)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7695 - loss: 1.0439
Test Accuracy: 79.04%
Test Loss: 1.0089614391326904
